# Pathwest-Source Date & Time Reconciliation

In [37]:
import csv
import json
import pprint
import datetime

## Create Mapping

Map pathwest csv file records to the csv file records of medrio.

In [24]:
randfilename = './test/rand.csv'

subjectMap = dict() # dictionary for randomisation to subject id.

randCol = 'DARandNum_C' # column header for randomisation number
subjectCol = 'Subject ID' # column header for subject id

with open(randfilename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        subjectMap[f'{row[randCol]}'] = row[subjectCol]

print(subjectMap)

{'14101': 'S007-14101', '14102': 'S008-14102', '14104': 'S009-14104', '14105': 'S010-14105', '14106': 'S012-14106', '14107': 'S013-14107', '14110': 'S015-14110', '14113': 'S019-14113', '14111': 'S020-14111', '14114': 'S021-14114', '14112': 'S024-14112', '14109': 'S025-14109', '14246': 'S058-14246'}


## Read Configuration File

Parse config file and establish relationships.

In [12]:
configfilename = './test/config.json'

configDict = dict()

with open(configfilename) as jsonfile:
    configDict = json.load(jsonfile)

print(configDict)

{'source': {'file': './INPUT.csv'}, 'comparison': {'file': './COMPARISON.csv'}, 'output': {'directory': './output'}, 'match': {'0': {'date': 'PKPDDat_C', 'time': 'PKPDTim_C'}, '0.25': {'date': '', 'time': ''}, '0.5': {'date': '', 'time': ''}, '1': {'date': '', 'time': ''}, '1.5': {'date': '', 'time': ''}, '2': {'date': '', 'time': ''}, '3': {'date': '', 'time': ''}, '4': {'date': '', 'time': ''}, '5': {'date': '', 'time': ''}, '6': {'date': '', 'time': ''}, '8': {'date': '', 'time': ''}, '12': {}, '16': {}, '48': {'date': '', 'time': ''}}}


## Read Medrio Source File

Create dictionary mapping for each subject id to allow for hash map search of data.

In [26]:
sourcefilename = './test/source.csv'

sourceMap = dict() # dictionary for subject data

subjectidcol = 'Subject ID'
reader

with open(sourcefilename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row[subjectidcol] not in sourceMap:
            sourceMap[f'{row[subjectidcol]}'] = dict()
        for key in row:
            if row[key] != '':
                sourceMap[f'{row[subjectidcol]}'][f'{key}'] = row.get(key, '')

pprint.pprint(sourceMap)

{'S007-14101': {'Form': 'Blood Collection EOS',
                'Group': 'Cohort 1',
                'LB24Dat_C': '12/11/2019',
                'LB24PK_C': 'Yes',
                'LB24Tim_C': '08:00',
                'LBEOSDatTScng_C': '12/16/2019',
                'LBEOSPK': 'Yes',
                'LBEOSTim_C': '11:14',
                'PK12Coll_C': 'Yes',
                'PK12Dat_C': '12/10/2019',
                'PK12Tim_C': '20:01',
                'PK15Coll_C': 'Yes',
                'PK15Dat_C': '12/10/2019',
                'PK15Tim_C': '08:15',
                'PK16Coll_C': 'Yes',
                'PK16Dat_C': '12/11/2019',
                'PK16Tim_C': '00:04',
                'PK1Coll_C': 'Yes',
                'PK1Dat_C': '12/10/2019',
                'PK1Tim_C': '09:00',
                'PK2Coll_C': 'Yes',
                'PK2Dat_C': '12/10/2019',
                'PK2Tim_C': '10:00',
                'PK30Coll_C': 'Yes',
                'PK30Dat_C': '12/10/2019',
             

## Read Comparison File

In [45]:
comparisonMap = dict()

comparisonfilename = './test/comparison.csv'

randomisationCol = 'Subject'
periodCol = 'Period'

scheduleCol = 'Scheduled time (hrs post dose)'
dateCol = 'Blood Sample date'
timeCol = 'Blood Sample time (24 hrs format)'

with open(comparisonfilename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row[randomisationCol] not in comparisonMap:
            comparisonMap[f'{row[randomisationCol]}'] = dict()
        if row[periodCol] not in comparisonMap[f'{row[randomisationCol]}']:
            comparisonMap[f'{row[randomisationCol]}'][f'{row[periodCol]}'] = dict()
        # UPDATE INFORMATION
        if row[dateCol] is '':
            continue
        comparisonMap[f'{row[randomisationCol]}'][f'{row[periodCol]}']['timepoint'] = row.get(scheduleCol, '')
        comparisonMap[f'{row[randomisationCol]}'][f'{row[periodCol]}']['date'] = datetime.datetime.strptime(row.get(dateCol), '%d-%b-%y')
        comparisonMap[f'{row[randomisationCol]}'][f'{row[periodCol]}']['time'] = row.get(timeCol, '')

pprint.pprint(comparisonMap)

{'': {'': {}},
 '14230': {'Period 1': {'date': datetime.datetime(2020, 2, 10, 0, 0),
                        'time': ' 08:10:00',
                        'timepoint': '288'},
           'Period 2': {'date': datetime.datetime(2020, 2, 17, 0, 0),
                        'time': ' 12:21:00',
                        'timepoint': '288'}},
 '14231': {'Period 1': {'date': datetime.datetime(2020, 2, 10, 0, 0),
                        'time': ' 08:10:00',
                        'timepoint': '288'},
           'Period 2': {'date': datetime.datetime(2020, 2, 17, 0, 0),
                        'time': ' 07:54:00',
                        'timepoint': '288'}},
 '14232': {'Period 1': {'date': datetime.datetime(2020, 2, 10, 0, 0),
                        'time': ' 08:20:00',
                        'timepoint': '288'},
           'Period 2': {'date': datetime.datetime(2020, 2, 17, 0, 0),
                        'time': ' 08:41:00',
                        'timepoint': '288'}},
 '14233': {'Period 1':

<>:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-45-34def1bf2d85>:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if row[dateCol] is '':


In [41]:
datetime.datetime.strptime

<function datetime.strptime>

In [42]:
date

NameError: name 'date' is not defined